In [3]:
%pwd

'd:\\GenAI project\\Doctor-assistant-chatbot\\research'

In [4]:
import os 
os.chdir("../")

In [5]:
%pwd

'd:\\GenAI project\\Doctor-assistant-chatbot'

In [6]:
from langchain.document_loaders import  PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [7]:
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [10]:
extracted_data= load_pdf_files("data")

In [9]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:

    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [11]:
minimal_docs=filter_to_minimal_docs(extracted_data)

In [ ]:
minimal_docs

In [13]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [14]:
texts_chunk=text_split(minimal_docs)
print(f"No. of chunks: {len(texts_chunk)} ")

No. of chunks: 5859 


In [ ]:
texts_chunk

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

In [18]:
vector = embedding.embed_query("Hello world")
vector

[-0.03447723388671875,
 0.031023165211081505,
 0.006734973751008511,
 0.026108991354703903,
 -0.039362046867609024,
 -0.16030244529247284,
 0.06692394614219666,
 -0.0064415112137794495,
 -0.04745052382349968,
 0.014758817851543427,
 0.07087530195713043,
 0.05552754923701286,
 0.019193369895219803,
 -0.026251282542943954,
 -0.01010951679199934,
 -0.02694047801196575,
 0.022307446226477623,
 -0.022226616740226746,
 -0.1496925950050354,
 -0.017493078485131264,
 0.007676276378333569,
 0.054352231323719025,
 0.0032544422429054976,
 0.03172587975859642,
 -0.0846213772892952,
 -0.029405953362584114,
 0.05159562826156616,
 0.04812408611178398,
 -0.0033148485235869884,
 -0.05827920883893967,
 0.04196930304169655,
 0.022210706025362015,
 0.128188818693161,
 -0.02233896777033806,
 -0.011656241491436958,
 0.06292831897735596,
 -0.032876331359148026,
 -0.09122606366872787,
 -0.031175393611192703,
 0.05269952118396759,
 0.047034814953804016,
 -0.0842030942440033,
 -0.030056195333600044,
 -0.02074482

In [19]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [20]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [21]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [22]:
pc

In [40]:
from pinecone import ServerlessSpec 

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [41]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [25]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

Add more data to the existing pinecode index

In [26]:
dswith = Document(
    page_content="freecodecamp is a youtube channel that provides tutorials on various topics.",
    metadata={"source": "Youtube"}
)

In [27]:
docsearch.add_documents(documents=[dswith])

['7cd5025e-5b65-42ee-88d7-126e24802d06']

In [28]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [29]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='9a3b0a1e-5edf-4593-a40a-06d5f6be4ec6', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='8506b602-370c-421e-bb48-126592dc9d03', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='2c722168-c9c0-47f8-9a16-a14f9d80ec07', metadata={'source': 'data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged wi

In [30]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-4o")

In [31]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [32]:
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [33]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "what is the Treatment of Acne?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "what dswithbappy?"})
print(response["answer"])